In [4]:
sc.stop()

sc = SparkContext()
hc = HiveContext(sc)




In [5]:
q1 = """
select line_item_id
from prod_odd_dsp.click
where ymd between '2017-02-10' and '2017-02-15'
 and utm_source='datamind_cc'
 limit 10
"""

hc.sql(q1).show()

+------------+
|line_item_id|
+------------+
|         413|
|         413|
|         447|
|         447|
|         447|
|         426|
|         426|
|         426|
|         426|
|         413|
+------------+



In [6]:
hc.sql('select 1 as a').show()

+---+
|  a|
+---+
|  1|
+---+



In [1]:
init_query = '''
CREATE TABLE `user_kposminin.cred_app_visits`(
  `phone_mobile` varchar(35), 
  `approve` int, 
  `call_ymd` string, 
  `id` string, 
  `urlfr` string, 
  `cnt` int, 
  `duration` int, 
  `avg_hour` int)
PARTITIONED BY ( 
  `ymd` string)
STORED AS RCFile
;

create table user_kposminin.cred_app_id_phone as
select 
  financial_application_rk, 
  retro_date, 
  mobile_telephone_no as phone_num,
  id,
  load_src,
  financial_product_type_cd, 
  default_flg
from
  user_kposminin.credit_app_default_2014_2016 c 
  inner join (
     select 
       uid_str as id,
       property_value as phone_num,
       load_src
     from
       prod_dds.md_uid_property 
     where
       property_cd = 'PHONE' 
      ) m on c.mobile_telephone_no = m.phone_num
;

'''


query = '''
insert overwrite 
 table user_kposminin.cred_app_visits partition(ymd)
select
 id
 ,urlfr
 ,cast(count(*) as int) cnt
 ,MIN(time) as first_visit
 ,cast(MAX(time) - MIN(time) as int) as duration
 ,cast(from_unixtime(cast(AVG(time) as Bigint), 'HH') AS int) as avg_hour
 ,max(current_timestamp()) as load_dttm
 ,max(ymd) ymd
from
 (
   select
    id
    ,cast(timestamp as Bigint) as time
    ,concat(parse_url(url, 'HOST'), '#', path_fr) as urlfr
    ,ymd
   from
    prod_odd.weblog
    inner join user_kposminin.
    LATERAL VIEW explode(split(parse_url(url, "PATH"), '/')) tt AS path_fr
   where
    ymd = '#ymd'
   ) t
group by
 id
 ,urlfr
'''

ymd_to_process = '2015-12-31'

print(query.replace('#ymd',ymd_to_process))



insert overwrite 
 table user_kposminin.cred_app_visits partition(ymd)
select
 id
 ,urlfr
 ,cast(count(*) as int) cnt
 ,MIN(time) as first_visit
 ,cast(MAX(time) - MIN(time) as int) as duration
 ,cast(from_unixtime(cast(AVG(time) as Bigint), 'HH') AS int) as avg_hour
 ,max(current_timestamp()) as load_dttm
 ,max(ymd) ymd
from
 (
   select
    id
    ,cast(timestamp as Bigint) as time
    ,concat(parse_url(url, 'HOST'), '#', path_fr) as urlfr
    ,ymd
   from
    prod_odd.weblog
    left semi join user_kposminin.
    LATERAL VIEW explode(split(parse_url(url, "PATH"), '/')) tt AS path_fr
   where
    ymd = '2015-12-31'
   ) t
group by
 id
 ,urlfr



In [2]:
sc.stop()